In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import datetime
from src.heikin_ashi import heikin_ashi
from src.indicators import supertrend, VWAP2, smc, wavetrend3d, atr
import pandas_ta as ta

In [3]:
auxusd = pd.read_csv("./data_directory/AUXUSD.csv")
auxusd['timestamp'] = pd.to_datetime(auxusd['timestamp'], unit='ms')
auxusd['timestamp'] = pd.to_datetime(auxusd['timestamp'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')))
auxusd.set_index('timestamp', inplace=True)
auxusd.drop_duplicates(inplace=True)

In [4]:
eurgbp = pd.read_csv("./data_directory/EURGBP.csv")
eurgbp['timestamp'] = pd.to_datetime(eurgbp['timestamp'], unit='ms')
eurgbp['timestamp'] = pd.to_datetime(eurgbp['timestamp'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')))
eurgbp.set_index('timestamp', inplace=True)
eurgbp.drop_duplicates(inplace=True)

In [3]:
forex_data = pd.read_csv("./data_directory/EURUSD_2021_2023.csv")
forex_data['timestamp'] = pd.to_datetime(forex_data['timestamp'], unit='ms')
forex_data['timestamp'] = pd.to_datetime(forex_data['timestamp'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')))
forex_data.set_index('timestamp', inplace=True)
forex_data.drop_duplicates(inplace=True)

In [6]:
eurusd = pd.read_csv("./data_directory/EURUSD_NEW.csv")
eurusd.set_index("Date", inplace=True)
eurusd.index = pd.to_datetime(eurusd.index)
eurusd.drop_duplicates(inplace=True)

In [7]:
# Der choCH wird hier programmiert

def choCH(data, length, band):
    chocolad = pd.Series(index=data.index)
    ob = smc(data, length, band, "30T")
    prevvalue = 0
    orderblocks = []
    i = 0
    if(band == 0):
        for index, values in ob.iteritems():
            if(values > 0 and values != prevvalue):
                prevvalue = values
                give = [i, values]
                orderblocks.append(give)
            i += 1
        for i, (candle, price) in enumerate(orderblocks):
            if price > orderblocks[i-2][1] and orderblocks[i-1][1] < orderblocks[i-2][1] and orderblocks[i-2][1] < orderblocks[i-3][1] and orderblocks[i-3][1] < orderblocks[i-4][1]:
                chocolad.iloc[candle] = price
    elif(band == 1):
        for index, values in ob.iteritems():
            if(values > 0 and values != prevvalue):
                prevvalue = values
                give = [i, values]
                orderblocks.append(give)
            i += 1
        for i, (candle, price) in enumerate(orderblocks):
            if price < orderblocks[i-2][1] and orderblocks[i-1][1] > orderblocks[i-2][1] and orderblocks[i-2][1] > orderblocks[i-3][1] and orderblocks[i-3][1] > orderblocks[i-4][1]:
                chocolad.iloc[candle] = price
    return chocolad
    





/var/folders/6k/qhxq2vds24v690rsqtznkwqm0000gn/T/ipykernel_28160/3134265937.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  chocolad = pd.Series(index=data.index)
/var/folders/6k/qhxq2vds24v690rsqtznkwqm0000gn/T/ipykernel_28160/3134265937.py:10: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, values in ob.iteritems():


In [8]:

for index, value in hello.iteritems():
    if value > 0:
        print(index)

2021-06-20 20:00:00
2021-07-05 04:00:00
2021-07-22 01:00:00
2021-10-14 05:30:00
2022-01-28 15:30:00
2022-02-15 17:30:00
2022-03-07 22:00:00
2022-04-11 06:30:00
2022-04-19 15:00:00
2022-04-28 16:00:00
2022-05-09 06:30:00
2022-05-16 03:00:00
2022-06-30 19:00:00
2022-09-22 15:00:00
2022-10-24 09:30:00
2022-11-29 08:00:00
2022-12-07 16:30:00
2023-01-09 22:00:00
2023-01-19 02:00:00
2023-02-20 00:30:00
2023-02-24 01:30:00


/var/folders/6k/qhxq2vds24v690rsqtznkwqm0000gn/T/ipykernel_28160/2810368415.py:1: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, value in hello.iteritems():
